# Multi Agent Systems

## General Imports

In [1]:
import numpy as np

## Agents

In [2]:
class Bot:
    def __init__(self, bal_per_round, init_balance=0):
        self.balance = init_balance
        self.bal_per_round = bal_per_round

    def pre_round(self, winner, win_amount):
        self.balance += self.bal_per_round
        if winner == 0:
            self.balance -= win_amount

    def play_round(self, winner, win_amount):
        pass

In [3]:
class AllInBot(Bot):
    def play_round(self, winner, win_amount):
        super().pre_round(winner, win_amount)
        return self.balance

In [4]:
class RandomBot(Bot):
    def play_round(self, winner, win_amount):
        super().pre_round(winner, win_amount)
        return np.random.randint(0, self.balance)

In [5]:
# todo agent 1

In [6]:
# todo agent 2

In [7]:
# ...

## Variables

In [8]:
bot_list = [
    # add bots from previous blocks
    AllInBot,
    RandomBot
]

N = len(bot_list)

## Util Classes

In [9]:
class Bid:
    def __init__(self, index, amount):
        self.index = index
        self.amount = amount

## Auction

In [10]:
import hashlib

def shuffle_order(num_list):
    n = len(num_list)
    hash_index = int(hashlib.sha1(str(num_list).encode()).hexdigest(), 16) % 24
    result = []
    for i in range(n, 0, -1):
        result.append(num_list[hash_index % i])
        del num_list[hash_index % i]
        hash_index //= i
    return result

In [11]:
def auction(agent_indexes, rounds, bal_per_round):
    agent_indexes = shuffle_order(agent_indexes)
    agents = [bot_list[i](bal_per_round) for i in agent_indexes]

    score = [0] * N
    total = [0] * N
    balance = [0] * N

    (prev_winner, prev_bid) = (-1, -1)
    for _round in range(rounds):
        bids = []
        for i in range(N):
            balance[i] += bal_per_round

            if prev_winner == i:
                prev_winner = 0
            elif prev_winner != -1 and prev_winner < i:
                prev_winner += 1

            bid = int(agents[i].play_round(prev_winner, prev_bid))
            if bid < 0 or bid > balance[i]:
                name = bot_list[agent_indexes[i]].__name__
                raise ValueError("Agent #%s tried to bid with an invalid amount (%d)" % (name, bid))
            bids.append(Bid(i, bid))

        bids.sort(reverse=True, key= lambda b: b.amount)

        winner = -1
        top_bid = bids[0].amount
        for i in range(1, N):
            if bids[i].amount == top_bid:
                winner = -1
                bids[i].amount = -1
                bids[i - 1].amount = -1
            else:
                if bids[i - 1].amount == -1:
                    winner = i
                    top_bid = bids[i].amount
                else:
                    winner = i - 1
                    break

        if winner == -1:
            (prev_winner, prev_bid) = (-1, -1)
        else:
            w_bid = bids[winner]
            (prev_bid, prev_winner) = (w_bid.amount, w_bid.index)
            score[agent_indexes[prev_winner]] += 1
            total[agent_indexes[prev_winner]] += prev_bid
            balance[prev_winner] -= prev_bid

    return score, total

In [12]:
def show_results(score, total):
    results = sorted(map(list, zip(score, total, bot_list)), key= lambda k: (-k[0], k[1]))

    for score, total, agent in results:
        print("%-20s Score: %-6d Total: %d" % (agent.__name__, score, total))

In [13]:
def simulate(experiments=10, rounds=10, bal_per_round=500, print_experiments=True):
    scores = []
    totals = []
    
    for _ in range(experiments):
        agent_indexes = list(range(N))
        (score, total) = auction(agent_indexes, rounds, bal_per_round)
        scores.append(np.array(score))
        totals.append(np.array(total))
        
        if print_experiments:
            print("End of experiment")
            show_results(score, total)
            print()
    
    print("End of simulation")
    score = sum(scores)
    total = sum(totals)
    show_results(score, total)

In [14]:
simulate(1000, 100, print_experiments=False)

End of simulation
AllInBot             Score: 56778  Total: 49755500
RandomBot            Score: 43173  Total: 49070723
